In [1]:
from datetime import datetime, date
import pandas as pd
import dew_gwdata as gd

db = gd.sageodata()

In [2]:
wells = db.wells_in_groups(["PADTH"])
DH_NO = "(" + ",".join([str(d) for d in wells.dh_no]) + ")"

In [11]:
df = db.water_levels_between_dates(
    wells, pd.Timestamp("2015-03-01"), pd.Timestamp("2018-06-01")
)
df.obs_date.min(), df.obs_date.max()

(Timestamp('2015-03-03 00:00:00'), Timestamp('2018-05-10 15:49:06'))

In [30]:
dx = df.groupby([df.obs_date.dt.year])["dtw", "swl", "rswl"].describe(percentiles=())

In [31]:
dx.index.names.index("obs_date")

0

In [24]:
dx.columns = [f"{x[0]}_{x[1]}" for x in dx.columns]
dx

dtw_count  dtw_mean   dtw_std  dtw_min  dtw_50%  dtw_max  \
well_id obs_date                                                             
GLE028  2015            4.0    3.6375  0.102103     3.51     3.64     3.76   
        2016            4.0    4.1200  0.081650     4.06     4.09     4.24   
        2017            4.0    3.2125  0.395000     2.62     3.41     3.41   
        2018            1.0    3.1200       NaN     3.12     3.12     3.12   
GLE034  2015            3.0    3.9200  0.131149     3.78     3.94     4.04   
...                     ...       ...       ...      ...      ...      ...   
PAR045  2018            1.0    3.4100       NaN     3.41     3.41     3.41   
PAR046  2015            0.0       NaN       NaN      NaN      NaN      NaN   
        2016            0.0       NaN       NaN      NaN      NaN      NaN   
        2017            0.0       NaN       NaN      NaN      NaN      NaN   
        2018            0.0       NaN       NaN      NaN      NaN      NaN   

                  swl_count  swl_mean   swl_std  swl_min  swl_50%  swl_max  \
well_id obs_date                                                             
GLE028  2015            4.0    3.5175  0.102103     3.39     3.52     3.64   
        2016            4.0    4.0000  0.081650     3.94     3.97     4.12   
        2017            4.0    3.0925  0.395000     2.50     3.29     3.29   
        2018            1.0    3.0000       NaN     3.00     3.00     3.00   
GLE034  2015            3.0    3.8700  0.131149     3.73     3.89     3.99   
...                     ...       ...       ...      ...      ...      ...   
PAR045  2018            1.0    3.1700       NaN     3.17     3.17     3.17   
PAR046  2015            0.0       NaN       NaN      NaN      NaN      NaN   
        2016            0.0       NaN       NaN      NaN      NaN      NaN   
        2017            0.0       NaN       NaN      NaN      NaN      NaN   
        2018            0.0       NaN       NaN      NaN      NaN      NaN   

                  rswl_count  rswl_mean  rswl_std  rswl_min  rswl_50%  \
well_id obs_date                                                        
GLE028  2015             4.0    32.7725  0.102103     32.65     32.77   
        2016             4.0    32.2900  0.081650     32.17     32.32   
        2017             4.0    33.1975  0.395000     33.00     33.00   
        2018             1.0    33.2900       NaN     33.29     33.29   
GLE034  2015             3.0    34.3900  0.131149     34.27     34.37   
...                      ...        ...       ...       ...       ...   
PAR045  2018             1.0    30.1100       NaN     30.11     30.11   
PAR046  2015             0.0        NaN       NaN       NaN       NaN   
        2016             0.0        NaN       NaN       NaN       NaN   
        2017             0.0        NaN       NaN       NaN       NaN   
        2018             0.0        NaN       NaN       NaN       NaN   

                  rswl_max  
well_id obs_date            
GLE028  2015         32.90  
        2016         32.35  
        2017         33.79  
        2018         33.29  
GLE034  2015         34.53  
...                    ...  
PAR045  2018         30.11  
PAR046  2015           NaN  
        2016           NaN  
        2017           NaN  
        2018           NaN  

[179 rows x 18 columns]

In [22]:
sql = f"""
 SELECT   dh.drillhole_no AS dh_no,
         dh.unit_no      AS unit_long,
         To_char(dh.map_100000_no)
                  || '-'
                  || To_char(dh.dh_seq_no) AS unit_hyphen,
         Trim(To_char(dh.obs_well_plan_code))
                  || Trim(To_char(dh.obs_well_seq_no, '000')) AS obs_no,
         dh.dh_name,
         dh.amg_easting      AS easting,
         dh.amg_northing     AS northing,
         dh.amg_zone         AS zone,
         dh.neg_lat_deg_real AS latitude,
         dh.long_deg_real    AS longitude,
         summ.aq_subaq       AS aquifer,
         wl.obs_date,
         wl.standing_water_level AS swl,
         wl.depth_to_water       AS dtw,
         wl.rswl,
         wl.pressure,
         wl.temperature,
         coalesce(wl.dry_ind, 'N') as dry_ind,
         wl.anomalous_ind AS anomalous_ind,
         wl.pumping_ind   AS pumping_ind,
         wl.measured_during,
         wl.data_source_code AS data_source,
         wl.comments,
         wl.created_by,
         wl.creation_date,
         wl.modified_by,
         wl.modified_date
FROM     dhdb.wa_water_level_vw wl
join     dhdb.dd_drillhole_vw dh
ON       wl.drillhole_no = dh.drillhole_no
join     dhdb.dd_drillhole_summary_vw summ
ON       wl.drillhole_no = summ.drillhole_no
WHERE    wl.drillhole_no IN {DH_NO}
AND      wl.series_type = 'T'
AND      wl.obs_date between to_date('2018-03-01 00:00:00', 'yyyy-mm-dd HH24:MI:SS') and to_date('2018-05-10 00:00:00', 'yyyy-mm-dd HH24:MI:SS')
ORDER BY dh_no,
         obs_date
"""

In [23]:
df = db.query(sql)

In [24]:
sorted(df.obs_date.unique())[::-1]

[numpy.datetime64('2018-05-09T16:25:53.000000000'),
 numpy.datetime64('2018-05-09T16:09:47.000000000'),
 numpy.datetime64('2018-05-09T15:45:31.000000000'),
 numpy.datetime64('2018-05-09T15:24:33.000000000'),
 numpy.datetime64('2018-05-09T14:51:37.000000000'),
 numpy.datetime64('2018-05-09T14:29:55.000000000'),
 numpy.datetime64('2018-05-09T14:10:23.000000000'),
 numpy.datetime64('2018-05-09T13:46:05.000000000'),
 numpy.datetime64('2018-05-09T13:17:23.000000000'),
 numpy.datetime64('2018-03-23T13:56:38.000000000'),
 numpy.datetime64('2018-03-23T13:50:33.000000000'),
 numpy.datetime64('2018-03-23T13:45:23.000000000'),
 numpy.datetime64('2018-03-23T13:24:09.000000000'),
 numpy.datetime64('2018-03-23T13:17:52.000000000'),
 numpy.datetime64('2018-03-23T13:12:41.000000000'),
 numpy.datetime64('2018-03-23T13:06:47.000000000'),
 numpy.datetime64('2018-03-23T12:43:54.000000000'),
 numpy.datetime64('2018-03-23T12:36:56.000000000'),
 numpy.datetime64('2018-03-23T12:27:54.000000000'),
 numpy.datet